In [ ]:
# Внимание!!! Важно, что бы файлы с данными и исполняемый файл находились в одной папке, 
# тогда пути к тестовым и тренировочным наборам будут содержать только имена файлов.
# 
# В пути к тренировочным и тестовым данным запрежается использовать абсалютную адресацию, 
# то есть адресацию, в которой присутствуют имена папок. Путь должен содержать только имя файла.
#
# Напоминание: под моделью машинного обучения понимаются все действия с исходными данными, 
# которые необходимо произвести, что бы сопоставить признаки целевому значению.

### Область работы 1 (библиотеки)

In [ ]:
# Данный блок в области 1 выполняется преподавателем
# 
# данный блок предназначен только для подключения необходимых библиотек
# запрещается подключать библиотеки в других блоках
#
# установка дополнительных библиотек размещается прямо здесь (обязательно закоментированы)
#
# pip install

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.experimental import enable_halving_search_cv
from IPython.display import display_html
from sklearn.model_selection import train_test_split 
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import roc_curve, plot_roc_curve, roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline , make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import RocCurveDisplay, DetCurveDisplay
from sklearn.metrics import PrecisionRecallDisplay
from sklearn.naive_bayes import GaussianNB, BernoulliNB,MultinomialNB
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler , Normalizer
from sklearn.preprocessing import PowerTransformer , QuantileTransformer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, LeaveOneOut
from sklearn.model_selection import HalvingGridSearchCV, KFold , RandomizedSearchCV, ShuffleSplit, StratifiedShuffleSplit
from sklearn.metrics import r2_score , accuracy_score, precision_score, recall_score ,log_loss,balanced_accuracy_score

In [ ]:
# необходимо для работы 3 блока

# функция для замены цитрусовых на 1 и 0
def zamena(df):
   return df.replace('grapefruit',1).replace('orange',0)

# нужно для 2 и 3 блока(это помогает обрезать данные с нужными колонками)
col = ['name','diameter','weight']
columns = ['diameter','weight']

### Область работы 2 (поиск лучшей модели)

In [ ]:
# Данный блок(и) НЕ выполняются преподавателем в области 2
# блок(и) предназначены для поиска лучшей модели
# 
# Запрещается размещать данные блоки за пределами обасти 2
# Все блоки данной области должны быть выполнены
#


# функция для замены названия фруктов на 1 или 0 
#def zamena(df):
#   return df.replace('grapefruit',1).replace('orange',0)

# Путь к тренировочному набору
path_train = 'cit_train.csv'
#для работы можно выбрать наиболее важные регрессоры, которые будут иметь больший вес с модели - вес и диаметер
# применяем функцию замены цитрусовых на 1 и о, а также обрезаем фрейм
df_raw= pd.read_csv(path_train).apply(zamena)[col]
# думаю, что можно глянуть дубликаты в обрезанном фрейме, чтобы они не мешали нам учиться
#display(df.duplicated().sum()) # видим 1542 дубликата => мы их убираем
df = df_raw.drop_duplicates(keep = 'last', ignore_index= True) #дропаем дубликаты
display(df)


In [ ]:
# можно глянуть конечно на данныt(но это нужно брать df_raw без обреза и без функции,)
plt.figure(figsize=(15,10))
sns.pairplot(df_raw, hue= 'name')

In [ ]:
# блок для простого анализа
X_train, X_test, y_train, y_test = train_test_split(df.drop(['name'],axis = 1), df['name'], test_size =0.3,random_state=42)



scaler = MinMaxScaler() # используем функцию масштабирования
scaler.fit(X_train)     # Перемасштабирование ТОЛЬКО на ТРЕНИРОВОЧНОМ наборе, создали правило перемасштабирования  

X_train = scaler.transform(X_train) # перемасштабировали тренировочный набор
X_test = scaler.transform(X_test)

knn = KNeighborsClassifier(n_neighbors = 2, p = 1, weights = 'uniform')
knn.fit(X_train, y_train)

sc_train = knn.score(X_train , y_train)
sc_test = knn.score(X_test , y_test)


print("тренировочный: {:.3f}".format(sc_train))
print("тестовый: {:.3f}".format(sc_test))

In [ ]:
# ниже есть через Log Regr отбор "значимых" фич, но это тоже эффективно использовать
# использовалась только для просмотра важности фич
# тут надо брать сырой сырой фрейм без обрезки
X_train, X_test, y_train, y_test = train_test_split(df_raw.drop(['name'],axis = 1), df_raw['name'], test_size =0.3, random_state=42)

# создаем случайное дерево
model = RandomForestClassifier(n_estimators=340, random_state= 42)

# Обучаем модель на вашей выборке
model.fit(X_train, y_train)

# Подбираем самые важные признаки
importances = model.feature_importances_
display(model.feature_importances_)

# Сет для визуализации
final_df = pd.DataFrame({"Features" : pd.DataFrame(X_train).columns, "Importances" : importances})
final_df.set_index('Importances')

# Сортируем их 
final_df = final_df.sort_values('Importances', ascending= False)

# Выводим на график
final_df.plot.bar(color = 'red')
display(final_df)
# то есть можно увидеть, что 2 фичи более значимы, чем все остальные
# можно и взять их для модели 

In [ ]:
# можно посмотреть значимость наших фич для этого фрейма,ну и выбрать по абсолютному значения наилучших для модели
# чтобы посмотреть корректно, то коментируем функцию и столбцы когда создаем врейм df_raw
X_train, X_test, y_train, y_test = train_test_split(df_raw.drop(['name'],axis = 1), df_raw['name'], test_size =0.3, random_state=42)

lg_model = Pipeline([('preprocessing',MinMaxScaler()), 
                 ('clf', LogisticRegression(penalty='l1', solver= 'liblinear', max_iter= 4500))])

lg_model.fit(X_train, y_train)
print(lg_model.score(X_train, y_train))
print(lg_model.score(X_test, y_test))
print(lg_model.named_steps['clf'].coef_)
# видно,что 1 и 2 фича будут значительно улучшать модель
#поэтому берем их 

## KNN


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(['name'],axis = 1), df['name'], test_size =0.3, random_state=42)

In [ ]:
# Создаем контейнер

pipe = Pipeline([
    ('preprocessing', MinMaxScaler()), 
    ('regressor',    KNeighborsClassifier())])


cv = KFold(n_splits = 10, shuffle= True)
#cv = ShuffleSplit(n_splits=15, )
#cv = KFold(n_splits = 10) 
#cv = LeaveOneOut()


# Параметры для решетки(можно менять и смотреть)
p = np.arange(1,6)           
n_neighbors = np.arange(1,16)
weights = ['uniform','distance']
scaling = [ MinMaxScaler(), StandardScaler(),RobustScaler(),Normalizer()]
param_grid = [ 
    {'preprocessing':scaling,
     'regressor': [KNeighborsClassifier()],
     'regressor__p':p,
     'regressor__n_neighbors': n_neighbors,
     'regressor__weights': weights}]
# есть много других,да это долго, но наиболее точно
grid = RandomizedSearchCV(pipe, param_grid, cv=cv, return_train_score=True, n_jobs=-1, scoring= 'accuracy')
grid.fit(X_train,y_train)
grid

In [ ]:
grid_result = pd.DataFrame(grid.cv_results_).sort_values(["rank_test_score",'std_test_score']).T
grid_result

In [ ]:
print("----------------- Обучили и тестировали -------------------")
print("Наилучшие параметры:\n{}\n".format(grid.best_params_))
print("Средняя правильность для наилучшей модели кроссвалидации на" 
      "валидационных тестовых наборах: {:.6f}\n".format(grid.best_score_)) 
print("Правильность для наилучшей модели на тестовом наборе: {:.6f}\n".format(grid.score(X_test, y_test)))

In [ ]:
# Подставляем параметры в контейнер
X_train, X_test, y_train, y_test = train_test_split(df.drop(['name'],axis = 1), df['name'], test_size =0.3, train_size=0.6,random_state=42)

finish_pipe1 =  Pipeline([
            ('preprocessing', MinMaxScaler()), 
            ('regressor',     KNeighborsClassifier(n_neighbors = 2, p = 1, weights = 'uniform'))
            ])
finish_pipe1.fit(X_train, y_train)

print(finish_pipe1.score(X_train,y_train))
print(finish_pipe1.score(X_test,y_test))

In [ ]:
# посмотреть для интереса как и что получается
y_predict = finish_pipe1.predict(X_test)
precision_score(y_predict, y_test)

### LOG Регресcия



In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(['name'],axis = 1), df['name'], test_size =0.3, random_state=42)


In [ ]:
pipe = Pipeline([('preprocessing', MinMaxScaler()), 
                 ('clf',           LogisticRegression(max_iter=4500))]) #max_iter=5000
#max_iter задаем, тк большой фрейм, а по стандарту стоит 100, а это очень мало и вылазит ошибка


cv = StratifiedShuffleSplit(n_splits = 10)
#cv = ShuffleSplit(n_splits = 5)
#cv = KFold(n_splits = 10)


scaling = [ MinMaxScaler(), StandardScaler(),RobustScaler(),Normalizer(),PowerTransformer(),QuantileTransformer()]

param_grid =[
    {'preprocessing': scaling,'clf__penalty': ['l2'], 
         'clf__solver': ['newton-cg' ,'lbfgs', 'liblinear', 'sag', 'saga']},
    {'preprocessing': scaling,'clf__penalty': ['l1'], 
         'clf__solver': ['liblinear']},
    {'preprocessing': scaling,'clf__penalty': ['none'], 
         'clf__solver': ['lbfgs','newton-cg']}
    ]

grid = GridSearchCV(pipe, param_grid, cv = cv, return_train_score = True) #scoring = 'accuracy')
grid.fit(X_train, y_train)
grid 

In [ ]:
grid_result = pd.DataFrame(grid.cv_results_).sort_values(["rank_test_score",'std_test_score']).T
grid_result

In [ ]:
print("Наилучшие параметры:\n{}\n".format(grid.best_params_))
print("Средняя правильность для наилучшей модели кроссвалидации на" 
      "валидационных тестовых наборах: {:.6f}\n".format(grid.best_score_)) 
print("Правильность для наилучшей модели на тестовом наборе: {:.6f}\n".format(grid.score(X_test, y_test)))

In [ ]:
# Подставляем параметры в контейнер
X_train, X_test, y_train, y_test = train_test_split(df.drop(['name'],axis = 1), df['name'], test_size =0.3,random_state=42)

finish_pipe2 =  make_pipeline( QuantileTransformer(), LogisticRegression(solver = 'lbfgs', penalty = 'none',max_iter=4500) )
finish_pipe2.fit(X_train, y_train)

print(finish_pipe2.score(X_train,y_train))
print(finish_pipe2.score(X_test,y_test))

In [ ]:
# посмотреть для интереса как и что получается
y_predict = finish_pipe2.predict(X_test)
accuracy_score(y_predict, y_test)

In [ ]:
# Ну тут можно подкрутить "С", но если penalty != none
# можно подкрутить модель

pipe_fin = make_pipeline(StandardScaler(), LogisticRegression(solver = 'liblinear', penalty = 'l1') )

from sklearn.model_selection import validation_curve

n_range = np.linspace(0.0001, 6, 500)
train_scores, test_scores = validation_curve(
    pipe_fin , X_train, y_train,
    param_name = "logisticregression__C", 
    param_range = n_range,
    cv = cv, 
    scoring="accuracy", 
    n_jobs=-1)

train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)

plt.figure(figsize=(15,10))

plt.title("Validation Curve")
plt.xlabel("сила регуляризации")
plt.ylabel("Score")
plt.ylim(0.7, 1.05)

plt.plot(n_range, train_scores_mean, label="Training score", color="darkorange")
plt.fill_between(n_range, train_scores_mean - train_scores_std,
                 train_scores_mean + train_scores_std, alpha=0.2,
                 color="darkorange")
plt.plot(n_range, test_scores_mean, label="Cross-validation score",
             color="navy")
plt.fill_between(n_range, test_scores_mean - test_scores_std,
                 test_scores_mean + test_scores_std, alpha=0.2,
                 color="navy")

plt.xticks(np.linspace(0, 6, 20))
plt.legend(loc="best")
plt.show()

In [ ]:
pipe_end = make_pipeline(StandardScaler(), LogisticRegression(solver = 'liblinear', penalty = 'l1', C = 2))
pipe_end.fit(X_train,y_train)
print(pipe_end.score(X_train,y_train))
print(pipe_end.score(X_test,y_test))

## Гаус


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(['name'],axis = 1), df['name'], test_size =0.3, random_state=42)


In [ ]:
pipe = Pipeline([('preprocessing', MinMaxScaler()), 
                 ('clf',           GaussianNB())])

#cv = StratifiedShuffleSplit(n_splits = 15)
#cv = ShuffleSplit(n_splits = 20)
cv = KFold(n_splits= 10 , shuffle= True)
#cv = LeaveOneOut()

scaling = [ MinMaxScaler(), StandardScaler(), RobustScaler(), Normalizer(), PowerTransformer(), QuantileTransformer()]

param_grid =[
    {'preprocessing': scaling, 
     'clf': [GaussianNB()]}
    ]

grid = GridSearchCV(pipe, param_grid, cv = cv, return_train_score = True)# scoring= 'accuracy')
grid.fit(X_train, y_train)
grid 

In [ ]:
grid_result = pd.DataFrame(grid.cv_results_).sort_values(["rank_test_score",'std_test_score']).T
grid_result

In [ ]:
print("Наилучшие параметры:\n{}\n".format(grid.best_params_))
print("Средняя правильность для наилучшей модели кроссвалидации на" 
      "валидационных тестовых наборах: {:.6f}\n".format(grid.best_score_)) 
print("Правильность для наилучшей модели на тестовом наборе: {:.6f}\n".format(grid.score(X_test, y_test)))

In [ ]:
# Подставляем параметры в контейнер
X_train, X_test, y_train, y_test = train_test_split(df.drop(['name'],axis = 1), df['name'], test_size =0.3,random_state=42)

finish_pipe3 = Pipeline(steps=[('preprocessing', Normalizer()), ('clf', GaussianNB())])
finish_pipe3.fit(X_train, y_train)

print(finish_pipe3.score(X_train,y_train))
print(finish_pipe3.score(X_test,y_test))


## Выбор модели 


In [ ]:
# СРАВНЕНИЕ 3 моделей
# 3 контейнера со своими лучшими моделям
X_train, X_test, y_train, y_test = train_test_split(df.drop(['name'],axis = 1), df['name'], test_size = .3, random_state=42)


model_NB = make_pipeline(Normalizer(),GaussianNB())
model_LR = make_pipeline(QuantileTransformer(),LogisticRegression(solver = 'lbfgs', penalty = 'none',max_iter=4500))
model_KNN = make_pipeline(MinMaxScaler(),KNeighborsClassifier(n_neighbors = 2, p=1, weights = 'uniform'))

model_NB.fit(X_train,y_train)
model_LR.fit(X_train,y_train)
model_KNN.fit(X_train,y_train)

y_NB_pred = model_NB.predict(X_test)
y_LR_pred = model_LR.predict(X_test)
y_KNN_pred = model_KNN.predict(X_test)

In [ ]:
# Сравниваем и выбираем лучшую модель
df1 = pd.DataFrame(classification_report(y_test, y_NB_pred, output_dict=True)).T
df2 = pd.DataFrame(classification_report(y_test, y_LR_pred, output_dict=True)).T
df3 = pd.DataFrame(classification_report(y_test, y_KNN_pred, output_dict=True)).T

df1_styler = df1.style.set_table_attributes("style='display:inline'").set_caption('GaussianNB')
df2_styler = df2.style.set_table_attributes("style='display:inline'").set_caption('LogisticRegression')
df3_styler = df3.style.set_table_attributes("style='display:inline'").set_caption('KNN')

display_html(df1_styler._repr_html_()+df2_styler._repr_html_()+df3_styler._repr_html_(), raw=True)

In [ ]:
print(roc_auc_score(y_test, finish_pipe1.predict_proba(X_test)[:,1]))
print(roc_auc_score(y_test, finish_pipe2.predict_proba(X_test)[:,1]))
print(roc_auc_score(y_test, finish_pipe3.predict_proba(X_test)[:,1]))

fig, ax_roc = plt.subplots(1,1, figsize=(10, 5))

RocCurveDisplay.from_estimator(finish_pipe1, X_test, y_test, ax = ax_roc, name = 'pipe_1');
RocCurveDisplay.from_estimator(finish_pipe2, X_test, y_test, ax = ax_roc, name = 'pipe_2');
RocCurveDisplay.from_estimator(finish_pipe3, X_test, y_test, ax = ax_roc, name = 'pipe_3');

In [ ]:
fig, [ax_roc, ax_det] = plt.subplots(1,2, figsize=(10, 5))

RocCurveDisplay.from_estimator(finish_pipe1, X_test, y_test, ax = ax_roc, name = 'finish_pipe_1');
RocCurveDisplay.from_estimator(finish_pipe2, X_test, y_test, ax = ax_roc, name = 'finish_pipe_2');
RocCurveDisplay.from_estimator(finish_pipe3, X_test, y_test, ax = ax_roc, name = 'finish_pipe_3');

DetCurveDisplay.from_estimator(finish_pipe1, X_test, y_test, ax = ax_det, name = 'finish_pipe_1');
DetCurveDisplay.from_estimator(finish_pipe2, X_test, y_test, ax = ax_det, name = 'finish_pipe_2');
DetCurveDisplay.from_estimator(finish_pipe3, X_test, y_test, ax = ax_det, name = 'finish_pipe_3');

In [ ]:
fig, ax_pr = plt.subplots(1,1, figsize=(10, 5))

PrecisionRecallDisplay.from_estimator(finish_pipe1, X_test, y_test, ax = ax_pr, name = 'finish_pipe_1');
PrecisionRecallDisplay.from_estimator(finish_pipe2, X_test, y_test, ax = ax_pr, name = 'finish_pipe_2');
PrecisionRecallDisplay.from_estimator(finish_pipe3, X_test, y_test, ax = ax_pr, name = 'finish_pipe_3');

### Область работы 3 (выполнение лучшей модели)

In [ ]:
# Данный блок(и) в области 3 выполняется преподавателем
#
# В области находится одна, единственная, итоговая модель машинного обучения с однозначными, 
# зафиксированными параметрами
#
# В данной области категорически запрещается искать, выбирать, улучшать, оптимизировать, 
# тюниговать и т.д. модель машинного обучения

In [ ]:
#zamena - это функция для замены цитрусовых на 1 и 0.
#
#все команды к df & df1 я описал во 2 блоке, поэтому сейчас я просто применю почти без пояснения
#


# Путь к тренировочному набору
path_train = 'cit_train.csv'    # содержит только имя файла, без имен папок
df_raw= pd.read_csv(path_train).apply(zamena)[col] # col из 1 блока(тут наши регрессоры) 
df = df_raw.drop_duplicates(keep = 'last', ignore_index= True) #дропаем дубликаты
display(df)

# Путь к тестовому набору
#тк это тест фрейм, то произведем пару манипуляций над ним,чтобы потом сопоставить с нужным
path_test  = 'cit_test.csv'  # содержит только имя файла, без имен папок
df1_raw = pd.read_csv(path_test)[columns] # тоже срезаем его по нужным колонкам
display(df1_raw)
ind = df1_raw[df1_raw[['diameter','weight']].duplicated(keep = 'last') == True].index #индексы,которые мы будем убирать 
df1 = df1_raw.drop(index = ind).reset_index(drop = True)
display(df1)

#zamena - это функция для замены цитрусовых на 1 и 0.

In [ ]:
# Блок(и) обучения и поверки модели

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(['name'],axis = 1), df['name'], test_size = 0.3)

finish_pipe =  Pipeline([
            ('preprocessing', MinMaxScaler()), 
            ('regressor',     KNeighborsClassifier(n_neighbors = 2, p = 1, weights = 'uniform'))
            ])
finish_pipe.fit(X_train, y_train)

print(finish_pipe.score(X_train,y_train))
print(finish_pipe.score(X_test,y_test))

In [ ]:
# Блок предсказания с использованием тестового набора

In [ ]:
y_predict = finish_pipe.predict(df1)
y_predict.shape

In [ ]:
# Название вектора предсказанных значений  y_predict полученого на основании тестового набора
y_predict = y_predict

## Подстановка


In [ ]:
column = ['name']
y_true_raw = pd.read_csv('.....').apply(zamena)[column] 
y_true = y_true_raw.drop(index = ind).reset_index(drop = True)